In [1]:
import pandas as pd

In [2]:
# importing census data for NYS household income
raw = pd.read_csv('../Datasets/ACSST5Y2020.S1901_dataset.csv',header=1)
raw.columns

Index(['id', 'Geographic Area Name', 'Estimate!!Households!!Total',
       'Margin of Error!!Households!!Total',
       'Estimate!!Households!!Total!!Less than $10,000',
       'Margin of Error!!Households!!Total!!Less than $10,000',
       'Estimate!!Households!!Total!!$10,000 to $14,999',
       'Margin of Error!!Households!!Total!!$10,000 to $14,999',
       'Estimate!!Households!!Total!!$15,000 to $24,999',
       'Margin of Error!!Households!!Total!!$15,000 to $24,999',
       ...
       'Estimate!!Nonfamily households!!Median income (dollars)',
       'Margin of Error!!Nonfamily households!!Median income (dollars)',
       'Estimate!!Nonfamily households!!Mean income (dollars)',
       'Margin of Error!!Nonfamily households!!Mean income (dollars)',
       'Estimate!!Nonfamily households!!PERCENT ALLOCATED!!Household income in the past 12 months',
       'Margin of Error!!Nonfamily households!!PERCENT ALLOCATED!!Household income in the past 12 months',
       'Estimate!!Nonfamily 

In [3]:
# median income per zip code in NYS
median_income = raw[['Geographic Area Name','Estimate!!Households!!Median income (dollars)']]
median_income.head()

,Geographic Area Name,Estimate!!Households!!Median income (dollars)
0,ZCTA5 06390,46250
1,ZCTA5 10001,96787
2,ZCTA5 10002,35607
3,ZCTA5 10003,129981
4,ZCTA5 10004,204949


In [4]:
# testing lambda function to get just zip code
a = 'ZCTA5 06390'
int(a.split('ZCTA5 ')[1])

6390

In [5]:
# converting zip codes into integers
# renaming columns
median_income['Geographic Area Name'] = median_income['Geographic Area Name'].apply(lambda a: int(a.split('ZCTA5 ')[1]))
median_income = median_income.rename(columns={'Geographic Area Name': 'ZipCode', 'Estimate!!Households!!Median income (dollars)':'MedianIncome'})
median_income.head()

C:\Users\Yiqing Guo\AppData\Local\Temp\ipykernel_19548\3991509314.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  median_income['Geographic Area Name'] = median_income['Geographic Area Name'].apply(lambda a: int(a.split('ZCTA5 ')[1]))


,ZipCode,MedianIncome
0,6390,46250
1,10001,96787
2,10002,35607
3,10003,129981
4,10004,204949


In [6]:
# importing school to zip code files
locations = pd.read_csv('../Datasets/grf21_lea_zcta5ce20.csv')
nyc_locations = pd.read_csv('../Datasets/nycdistrict-zipcode.csv')

In [7]:
locations.head()

,LEAID,NAME_LEA21,ZCTA5CE20,COUNT,LANDAREA,WATERAREA
0,100001,Fort Rucker School District,0,4,38.768437,1.038161
1,100090,Anniston City School District,0,7,3.851023,0.013616
2,100270,Baldwin County School District,0,24,131.184301,358.000989
3,100360,Bibb County School District,0,12,35.795340,0.004291
4,100540,Calhoun County School District,0,14,63.066080,0.164629


In [8]:
# just school district to zip code
districtZCTA = locations[['NAME_LEA21','ZCTA5CE20']]

# filter for just NYS zip codes
districtZCTA = districtZCTA[(districtZCTA['ZCTA5CE20']==6390) | ((districtZCTA['ZCTA5CE20']>=10001) & (districtZCTA['ZCTA5CE20']<=14905))]

# exclude NYCDOE on district name because it does not show specific districts in NYC
districtZCTA = districtZCTA[districtZCTA['NAME_LEA21'] != 'New York City Department Of Education']

# rename columns for merging
districtZCTA = districtZCTA.rename(columns={'NAME_LEA21':'DistrictName','ZCTA5CE20':'ZipCode'})

# uppercase for naming conventions
districtZCTA['DistrictName'] = districtZCTA['DistrictName'].apply(lambda x: x.upper())

In [9]:
# dictionary for mapping nyc school districts to their numbers in the nyc to zip code dataset
districtnamemap = {1:'NYCGEOGDIST#1-MANHATTAN',
 2:'NYCGEOGDIST#2-MANHATTAN',
 3:'NYCGEOGDIST#3-MANHATTAN',
 4:'NYCGEOGDIST#4-MANHATTAN',
 5:'NYCGEOGDIST#5-MANHATTAN',
 6:'NYCGEOGDIST#6-MANHATTAN',
 7:'NYCGEOGDIST#7-BRONX',
 8:'NYCGEOGDIST#8-BRONX',
 9:'NYCGEOGDIST#9-BRONX',
 10:'NYCGEOGDIST#10-BRONX',
 11:'NYCGEOGDIST#11-BRONX',
 12:'NYCGEOGDIST#12-BRONX',
 13:'NYCGEOGDIST#13-BROOKLYN',
 14:'NYCGEOGDIST#14-BROOKLYN',
 15:'NYCGEOGDIST#15-BROOKLYN',
 16:'NYCGEOGDIST#16-BROOKLYN',
 17:'NYCGEOGDIST#17-BROOKLYN',
 18:'NYCGEOGDIST#18-BROOKLYN',
 19:'NYCGEOGDIST#19-BROOKLYN',
 20:'NYCGEOGDIST#20-BROOKLYN',
 21:'NYCGEOGDIST#21-BROOKLYN',
 22:'NYCGEOGDIST#22-BROOKLYN',
 23:'NYCGEOGDIST#23-BROOKLYN',
 24:'NYCGEOGDIST#24-QUEENS',
 25:'NYCGEOGDIST#25-QUEENS',
 26:'NYCGEOGDIST#26-QUEENS',
 27:'NYCGEOGDIST#27-QUEENS',
 28:'NYCGEOGDIST#28-QUEENS',
 29:'NYCGEOGDIST#29-QUEENS',
 30:'NYCGEOGDIST#30-QUEENS',
 31:'NYCGEOGDIST#31-STATENISLAND',
 32:'NYCGEOGDIST#32-BROOKLYN'}

nyc_locations['District'] = nyc_locations['District'].apply(lambda x: districtnamemap[x])
nyc_locations = nyc_locations.rename(columns={'District':'DistrictName'})

In [10]:
# combine both datasets
concatenated = pd.concat([districtZCTA, nyc_locations])
concatenated

,DistrictName,ZipCode
5050,FISHERS ISLAND UNION FREE SCHOOL DISTRICT,6390
6921,YONKERS CITY SCHOOL DISTRICT,10470
6927,SOMERS CENTRAL SCHOOL DISTRICT,10501
6928,ARDSLEY UNION FREE SCHOOL DISTRICT,10502
6929,IRVINGTON UNION FREE SCHOOL DISTRICT,10503
...,...,...
274,NYCGEOGDIST#29-QUEENS,11429
275,NYCGEOGDIST#29-QUEENS,11412
276,NYCGEOGDIST#29-QUEENS,11411
277,NYCGEOGDIST#29-QUEENS,11413


In [11]:
# merge datasets for district and median income
district_medianincome = concatenated.merge(median_income, on='ZipCode', how='left')

# convert all values into numeric, some of them are strings...
district_medianincome['MedianIncome'] = pd.to_numeric(district_medianincome['MedianIncome'], errors='coerce')

In [12]:
# group by district name, find average median income
district = district_medianincome.groupby("DistrictName")['MedianIncome'].mean()

In [13]:
# export
district = district.to_frame()
district.to_csv('../Datasets/medianincomeall.csv')